# Вариант 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    Наименование вакансии.
    Предлагаемую зарплату (отдельно минимальную и максимальную).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия. ### По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

##  Подгружаем библиотеки

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import json
import lxml
import csv
import pandas as pd
import urllib.parse
from fake_headers import Headers

## Запишем headers 

In [2]:
HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0'
}

## HeadHanter

In [3]:
URL = 'https://hh.ru/search/vacancy'

### Получаем html страницы

In [4]:
def get_html(url, params=''):
    html = req.get(url, headers=HEADERS, params=params)
    return html

### Получаем контент со страницы html

In [5]:
def get_hh_content(html):
    soup = bs(html, 'lxml')
    items = soup.find_all('div', class_='vacancy-serp-item')
    vacancy = []

    for item in items:
        vacancy.append(
            {
                'site': 'HeadHanter', # Название сайта
                'title': item.find('div', class_='vacancy-serp-item__info').get_text(), # Название вакансии
                'link': item.find('div', class_='vacancy-serp-item__info').find('a').get('href'), # Ссылка на вакансию
                'salary': item.find('div', class_='vacancy-serp-item__sidebar').get_text(), # Зарплата
                'city': item.find('span', class_='vacancy-serp-item__meta-info').get_text(), # Город
                'organization': item.find('div', class_='vacancy-serp-item__meta-info-company').get_text(), # Название компании
                'note': item.find('div', class_='vacancy-label') # Примечание
            }
        )      
    # Приведем данные к нормальному виду
    for i in vacancy:
        # Salary
        if i['salary']:
            salary_list = i['salary'].split(' ')
            if salary_list[0] == 'от':
                i['salary_min'] = salary_list[1]
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = salary_list[1]
            else:
                i['salary_min'] = salary_list[0]   
                i['salary_max'] = salary_list[2]     
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None  
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
        # note
        if i['note'] != None:
            i['note'] = i['note'].get_text()
        # City
        if i['city']:
            city_list = i['city'].split(',')
            i['city'] = city_list[0]
    return vacancy

### Главная функция 

## SuperJob 

In [6]:
HOST = 'https://superjob.ru' # Нужен для получения полных ссылок на вакансии.
URL = 'https://superjob.ru/vacancy/search/'

### Получаем контент со страницы 

In [7]:
def superjob_get_content(html):
    soup = bs(html, 'lxml')
    items = soup.find_all('div', class_='f-test-vacancy-item')
    
    vacancy = []
    for item in items:       
         vacancy.append(
            {
                'site': 'SuperJob', # Название сайта
                'title': item.find('a').get_text(), # Название вакансии
                'link': item.find('a').get('href'), # Ссылка на вакансию
                'salary': item.find('span', class_='f-test-text-company-item-salary').get_text(), # Зарплата
                'city': item.find('span', class_='f-test-text-company-item-location').get_text(), # Город
                'organization': item.find('span', class_='f-test-text-vacancy-item-company-name').get_text(), # Название компании
                'note': item.find('span', class_='f-test-badge') # Примечание
            }
        )
    # Почистим данные       
    for v in vacancy:
        # link
        v['link'] = HOST + v['link'] 
        #salary
        if v['salary'] != 'По договорённости':
            salary_list = v['salary'].split('\xa0')
            if salary_list[0] == 'от':
                v['salary_min'] = salary_list[1] + salary_list[2]
                v['salary_max'] = None
            elif salary_list[0] == 'до':
                v['salary_min'] = None
                v['salary_max'] = salary_list[1] + salary_list[2]
            elif len(salary_list) == 3:
                v['salary_min'] = salary_list[0] + salary_list[1]
                v['salary_max'] = salary_list[0] + salary_list[1]                
            else:
                v['salary_min'] = salary_list[0] + salary_list[1]   
                v['salary_max'] = salary_list[3] + salary_list[4]    
            v['salary_currency'] = salary_list[-1].split('/')[0]
        else:
            v['salary_min'] = None  
            v['salary_max'] = None
            v['salary_currency'] = None
        v.pop('salary')  
        # note
        if v['note'] != None:
            v['note'] = v['note'].get_text()
        # city
        city_split = v['city'].split(' ')
        if len(city_split[2]) >= 3:
            v['city'] = city_split[2]
        else:
            v['city'] = city_split[3]
    return vacancy

### Главная функция 

## Объединим два парсинг двух сайтов в одну функцию 

In [8]:
def main_parsing():
    HH_URL = 'https://hh.ru/search/vacancy'
    SJ_HOST = 'https://superjob.ru'
    SJ_URL = 'https://superjob.ru/vacancy/search/'
    
    POST = str(input('Введите название вакансии для парсинга: '))
    PAGES = int(input('Количество страниц для парсинга: '))
    HH_HTML = get_html(HH_URL)
    SJ_HTML = get_html(SJ_URL)
    
    if HH_HTML.status_code == 200 and SJ_HTML.status_code == 200:
        vacancy = []
        for page in range(1, PAGES+1):
            print(f'Парсятся страницы {page}')
            hh = get_html(HH_URL, params={'text': POST, 'page': page})
            sj = get_html(SJ_URL, params={'keywords': POST, 'page': page})
            vacancy.extend(get_hh_content(hh.text) + superjob_get_content(sj.text))
        result = pd.DataFrame(vacancy)
    else:
        print('error')
    return result

In [9]:
df = main_parsing()
df

Введите название вакансии для парсинга:  python
Количество страниц для парсинга:  5


Парсятся страницы 1
Парсятся страницы 2
Парсятся страницы 3
Парсятся страницы 4
Парсятся страницы 5


,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
0,HeadHanter,Python разработчик,https://perm.hh.ru/vacancy/46154512?from=vacan...,Москва,idaproject,None,70 000,None,руб.
1,HeadHanter,ML инженер / Data scientist,https://perm.hh.ru/vacancy/46754459?from=vacan...,Москва,ООО КИБЕР-РОМ,None,250 000,350 000,руб.
2,HeadHanter,Программист-секретарь Python/JavaScript,https://perm.hh.ru/vacancy/43244378?from=vacan...,Москва,ООО ИнфоТех,None,60 000,None,руб.
3,HeadHanter,Программист Python,https://perm.hh.ru/vacancy/46539766?from=vacan...,Москва,ООО Артистраж,None,60 000,200 000,руб.
4,HeadHanter,Разработчик Python / Python Developer,https://perm.hh.ru/vacancy/46823068?from=vacan...,Москва,Gromik group,None,80 000,120 000,руб.
...,...,...,...,...,...,...,...,...,...
100,HeadHanter,Python developer,https://perm.hh.ru/vacancy/45728419?from=vacan...,Барнаул,ООО АйТи Мегастар,Будьте первыми,80 000,130 000,руб.
101,HeadHanter,Инженер - программист 2 категории PYTHON (MIDD...,https://perm.hh.ru/vacancy/44317825?from=vacan...,Москва,"АО Моринформсистема-Агат, Концерн",Будьте первыми,100 000,None,руб.
102,HeadHanter,Python разработчик,https://perm.hh.ru/vacancy/46902860?from=vacan...,Уфа,Technoled,Будьте первыми,None,None,None
103,HeadHanter,Python Django программист,https://perm.hh.ru/vacancy/46860014?from=vacan...,Москва,ООО География Турс,Будьте первыми,150 000,None,руб.


In [10]:
df.loc[df['site'] == 'SuperJob'].head(5)

,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
20,SuperJob,Python Developer,https://superjob.ru/vakansii/python-developer-...,Пермь,Onega Global Recruitment,Удаленная работа,None,250000,руб.
21,SuperJob,Специалист службы поддержки (с техническими зн...,https://superjob.ru/vakansii/specialist-sluzhb...,Пермь,Яндекс,Опыт не нужен,15000,None,руб.
22,SuperJob,Программист 1С,https://superjob.ru/vakansii/programmist-1s-36...,Пермь,СКБ Контур,Удаленная работа,None,None,None
23,SuperJob,Data engineer,https://superjob.ru/vakansii/data-engineer-382...,Пермь,ВИЗАВИ Консалт,Удаленная работа,None,None,None
24,SuperJob,"Backend-разработчик (PHP, Middle)",https://superjob.ru/vakansii/backend-razrabotc...,Пермь,Технопарк «Сколково»,None,110000,None,руб.


In [11]:
df.loc[df['site'] == 'HeadHanter'].head(5)

,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
0,HeadHanter,Python разработчик,https://perm.hh.ru/vacancy/46154512?from=vacan...,Москва,idaproject,None,70 000,None,руб.
1,HeadHanter,ML инженер / Data scientist,https://perm.hh.ru/vacancy/46754459?from=vacan...,Москва,ООО КИБЕР-РОМ,None,250 000,350 000,руб.
2,HeadHanter,Программист-секретарь Python/JavaScript,https://perm.hh.ru/vacancy/43244378?from=vacan...,Москва,ООО ИнфоТех,None,60 000,None,руб.
3,HeadHanter,Программист Python,https://perm.hh.ru/vacancy/46539766?from=vacan...,Москва,ООО Артистраж,None,60 000,200 000,руб.
4,HeadHanter,Разработчик Python / Python Developer,https://perm.hh.ru/vacancy/46823068?from=vacan...,Москва,Gromik group,None,80 000,120 000,руб.


In [12]:
df.to_csv('09082021', index=False)

#  Единственная большая проблема, которую не смог решить - на сайте SuperJob парсятся вакансии только из моего города. Хотя нигдя я такие параметры не указывал. Как исправить не знаю.
